# Word Embedding
- CBoW

In [426]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

torch.manual_seed(1)

In [343]:
sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


vocab = list(set(sentence))
vocab_size = len(vocab)
vocab_size

97

In [344]:
data = \
[([sentence[idx-2],sentence[idx-1],sentence[idx+1],sentence[idx+2]],sentence[idx])\
 for idx in range(2,len(sentence)-2)]
print(data[:3],end='\n\n\n')

word_to_ix = {val : idx for idx,val in enumerate(vocab)}
print(word_to_ix)

[(['When', 'forty', 'shall', 'besiege'], 'winters'), (['forty', 'winters', 'besiege', 'thy'], 'shall'), (['winters', 'shall', 'thy', 'brow,'], 'besiege')]


{'praise.': 0, 'a': 1, 'besiege': 2, 'it': 3, 'deep': 4, 'be': 5, 'lies,': 6, 'field,': 7, 'sunken': 8, 'made': 9, 'all': 10, 'proud': 11, 'in': 12, 'To': 13, "beauty's": 14, 'Proving': 15, 'treasure': 16, 'my': 17, 'see': 18, 'livery': 19, 'eyes,': 20, "'This": 21, 'sum': 22, 'forty': 23, 'use,': 24, 'own': 25, 'praise': 26, 'old,': 27, 'brow,': 28, 'so': 29, 'say,': 30, 'blood': 31, "excuse,'": 32, 'gazed': 33, 'When': 34, 'worth': 35, 'thou': 36, 'old': 37, 'make': 38, 'were': 39, 'his': 40, 'more': 41, 'small': 42, 'count,': 43, 'thy': 44, 'mine': 45, 'succession': 46, 'asked,': 47, 'beauty': 48, 'Then': 49, 'fair': 50, "feel'st": 51, 'by': 52, 'within': 53, 'held:': 54, 'couldst': 55, 'now,': 56, 'an': 57, 'shame,': 58, 'And': 59, 'the': 60, 'when': 61, 'to': 62, 'child': 63, 'lusty': 64, 'much': 65, 'Will': 66, 'shall': 67, '

In [385]:
class CBOW(nn.Module) : 
    
    def __init__(self,vocab_size, embedding_dim):  
        
        super(CBOW,self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_size
        self.embeddings = nn.Embedding(self.vocab_size,self.embedding_dim)
        self.linear1 = nn.Linear(self.embedding_dim,128)
        self.linear2 = nn.Linear(128,self.vocab_size)
    
    def forward(self,inputs) : 
        self.embeds = self.embeddings(inputs).sum(dim=0).unsqueeze(0)
        out = F.relu(self.linear1(self.embeds))
        out = self.linear2(out)
            
        log_probs = F.log_softmax(out,dim=1)
        return log_probs
    
    def get_word_vector(self,target,word_to_ix) : 
        word2id = torch.LongTensor([word_to_ix[target]])
        return self.embeddings(word2id).view(1, -1)
        
    
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    word2id = torch.tensor(idxs, dtype=torch.long)
    return word2id

def make_target_vector(target, word_to_ix):
    idxs = [word_to_ix[target]]
    return torch.LongTensor(idxs)

In [386]:
EMBEDDING_DIM = 10
EPOCH = 20
VERVOSE = 5

In [387]:
def run_cbow(vocab_size , word_to_ix) :
    loss_function = nn.NLLLoss()
    model = CBOW(vocab_size, EMBEDDING_DIM)
    optimizer = optim.SGD(model.parameters(), lr=0.001)

    for epoch in range(EPOCH):
        total_loss = 0
        for context, target in data:

            # 아래의 프로세스는 word : ID 로 converting 시켜주는 process 이다.
            context_idxs = make_context_vector(context,word_to_ix)
            target_idxs = torch.LongTensor([word_to_ix[target]])

            # pytorch는 gradient 를 누진적으로 계산하기 때문에, 0으로 만들어주어야한다.
            model.zero_grad()

            # input 값을 넣으면 log_probs 라는 output 값이 나온다.
            log_probs = model(context_idxs)

            # 이 값을 위에서 정의한 손실 함수를 기반으로 loss 를 계산한다.
            loss = loss_function(log_probs, target_idxs)

            # 위에서 나온 loss를 기반으로 back propagation 을 돌린다.
            # 또한, optimizer 를 update 하면서 parameter 또한 update 한다.
            loss.backward()
            optimizer.step()
            total_loss += loss

            if epoch % VERVOSE == 0 : 
                loss_avg = float(total_loss / len(data))
                print("{}/{} loss {:.2f}".format(epoch, EPOCH, loss_avg))
    return model

In [388]:
run_cbow(vocab_size , word_to_ix)

0/20 loss 0.05
0/20 loss 0.09
0/20 loss 0.14
0/20 loss 0.17
0/20 loss 0.21
0/20 loss 0.25
0/20 loss 0.29
0/20 loss 0.34
0/20 loss 0.39
0/20 loss 0.43
0/20 loss 0.47
0/20 loss 0.51
0/20 loss 0.55
0/20 loss 0.59
0/20 loss 0.63
0/20 loss 0.67
0/20 loss 0.71
0/20 loss 0.75
0/20 loss 0.79
0/20 loss 0.83
0/20 loss 0.87
0/20 loss 0.91
0/20 loss 0.95
0/20 loss 0.99
0/20 loss 1.04
0/20 loss 1.09
0/20 loss 1.13
0/20 loss 1.18
0/20 loss 1.22
0/20 loss 1.27
0/20 loss 1.31
0/20 loss 1.35
0/20 loss 1.39
0/20 loss 1.43
0/20 loss 1.48
0/20 loss 1.52
0/20 loss 1.56
0/20 loss 1.60
0/20 loss 1.64
0/20 loss 1.69
0/20 loss 1.73
0/20 loss 1.78
0/20 loss 1.80
0/20 loss 1.84
0/20 loss 1.88
0/20 loss 1.93
0/20 loss 1.96
0/20 loss 2.01
0/20 loss 2.05
0/20 loss 2.09
0/20 loss 2.13
0/20 loss 2.17
0/20 loss 2.21
0/20 loss 2.26
0/20 loss 2.30
0/20 loss 2.34
0/20 loss 2.39
0/20 loss 2.43
0/20 loss 2.47
0/20 loss 2.51
0/20 loss 2.56
0/20 loss 2.60
0/20 loss 2.64
0/20 loss 2.68
0/20 loss 2.72
0/20 loss 2.77
0/20 loss 

CBOW(
  (embeddings): Embedding(97, 5)
  (linear1): Linear(in_features=5, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=97, bias=True)
)

In [468]:
model = CBOW(vocab_size,EMBEDDING_DIM)

def test_cbow(model , vocab , word_to_ix) :
    word_1 = vocab[10]
    word_2 = vocab[5]
    
    word_1_vec = model.get_word_vector(word_1,word_to_ix)
    word_2_vec = model.get_word_vector(word_2,word_to_ix)
    
    cosine_similarity = (torch.mm(word_1_vec, word_2_vec.transpose(0,1))) / (torch.norm(word_1_vec) * torch.norm(word_2_vec))
    similarity = cosine_similarity.data.numpy()[0][0]
    print('word1 : ',word_1)
    print('word2 : ',word_2)
    print('similarity : ',similarity)

In [469]:
test_cbow(model,vocab,word_to_ix)

word1 :  all
word2 :  be
similarity :  -0.68877524
